# Was ist und kann Transformers?
Kurze Einführung und Vorstellung der Funktionen des transformers python packages

In [17]:
# Installation des Pakets (Nicht notwendig bei Google Colab)
!pip install pandas transformers torchaudio pydub


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


## Transkription von Audiodateien

In [21]:
from transformers import pipeline

# Neue Pipeline erstellen und Aufgabe zuweisen

# transcriber = pipeline(task="automatic-speech-recognition")

transcriber = pipeline(model="openai/whisper-large-v2")

# transcriber("test.mp3")

transcriber("test.mp3")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/Users/davidjosiger/Desktop/dev/hands-on-msc-journalismus/venv/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_k

{'text': ' für unser Land zu treffen. Darum ging es mir in den vergangenen drei Jahren. Darum geht es mir jetzt. Ich habe dem Koalitionspartner von der FDP heute Mittag noch einmal einen Umfang'}

## Testdaten

In [22]:
!pip install datasets

  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.9 MB/s eta 0:00:0000:01
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.9/390.9 kB 18.1 MB/s eta 0:00:00
Using cached multiprocess-0.70.16-py312-none-any.whl (146 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 22.6 MB/s eta 0:00:0000:0100:01
Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 7.2 

In [24]:
from datasets import load_dataset

# Datensätze laden und untersuchen
imdb = load_dataset("imdb")

imdb["test"][0]

Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 667398.19 examples/s]


{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

# Analyse von Nachrichtenartikeln
In diesem Abschnitt verwenden wir verschiedene NLP-Tools von Hugging Face, um einen großen Datensatz voller Spiegel-Artikel zu analysieren. Mit Hilfe von Transformer-Modellen führen wir Sentiment-Analyse, Named Entity Recognition (NER) und Topic Modeling durch, um wichtige Einblicke in die Artikel zu gewinnen.

### Ziel:
- Laden und Verarbeiten großer JSON-Datensätze in Chunks
- Durchführung von Sentiment-Analyse und NER auf Artikeldaten
- Visualisierung und Analyse der Ergebnisse

**Benötigte Zeit:** ~2 Stunden


## 1. Setup und Laden der JSON-Daten in Chunks
Wir beginnen mit der Installation der notwendigen Bibliotheken und laden die JSON-Daten in verwaltbaren Chunks, damit wir große Dateien effizient verarbeiten können.

In [ ]:
# Bibliotheken importieren
import json
import pandas as pd
from transformers import pipeline

# JSON-Daten in Chunks laden
def load_json_from_github_in_chunks(url, chunk_size=1000):
    response = requests.get(url)
    data = response.json()
    articles = list(data.values())
    for i in range(0, len(articles), chunk_size):
        yield articles[i:i + chunk_size]

# Pfad zur Datei angeben (Pfad entsprechend in Google Colab anpassen oder direkt von URL ziehen)
github_url = 'https://raw.githubusercontent.com/dajo51/hands-on-msc-journalismus/main/data_2009.json'

# Beispiel-Chunks laden und Struktur inspizieren
sample_chunk = next(load_json_from_github_in_chunks(github_url))
df_sample = pd.DataFrame(sample_chunk)
df_sample.head()

## 2. Extrahieren und Verarbeiten der Schlüsselinformationen
Wir transformieren die JSON-Daten in ein DataFrame-Format, um sie leichter analysieren zu können. Hier extrahieren wir Felder wie `title`, `description`, `text` und `keywords`.

In [ ]:

# Funktion zum Extrahieren der Schlüssel-Felder
def extract_fields(article_chunk):
    data = []
    for article in article_chunk:
        data.append({
            "title": article.get("title"),
            "description": article.get("description"),
            "text": article.get("text"),
            "keywords": article.get("keywords"),
            "author": article.get("author"),
            "date": article.get("date")
        })
    return pd.DataFrame(data)

# Beispiel-Chunks verarbeiten
df_articles = extract_fields(sample_chunk)

# Test Datensatz erstellen
test_df = df_articles.sample(40)

test_df.head()
    

## 3. Sentiment-Analyse auf Artikelbeschreibungen
Wir verwenden ein Transformer-Modell, das speziell für die deutsche Sprache trainiert wurde, um die Stimmung der Artikelbeschreibungen zu analysieren.

In [ ]:

# Sentiment-Analyse Pipeline mit einem deutschen Modell initialisieren
sentiment_analyzer = pipeline("sentiment-analysis", model="oliverguhr/german-sentiment-bert")

# Sentiment-Analyse auf die Beschreibungen anwenden
df_articles['sentiment'] = df_articles['description'].apply(lambda x: sentiment_analyzer(x)[0]['label'] if pd.notnull(x) else None)

# Ergebnisse anzeigen
df_articles[['title', 'description', 'sentiment']].head()
    

## 4. Named Entity Recognition (NER) auf Artikeltexten
Mit Hilfe eines multilingualen NER-Modells analysieren wir die wichtigsten Entitäten (z. B. Personen, Organisationen, Orte), die in den Artikeln genannt werden.

In [ ]:

# NER-Pipeline mit einem multilingualen Modell für die deutsche Sprache
ner_analyzer = pipeline("ner", model="Davlan/xlm-roberta-base-ner-hrl", grouped_entities=True)

# NER auf den Artikeltext anwenden
df_articles['entities'] = df_articles['text'].apply(lambda x: ner_analyzer(x) if pd.notnull(x) else None)

# Ergebnisse anzeigen
df_articles[['title', 'entities']].head()
    

## 5. Keyword-Extraktion aus dem Keyword-Feld
Wir analysieren die häufig vorkommenden Schlagwörter, um wiederkehrende Themen zu identifizieren.

In [ ]:

from collections import Counter

# Schlagwörter splitten und Häufigkeiten zählen
all_keywords = df_articles['keywords'].dropna().str.split(', ').sum()
keyword_counts = Counter(all_keywords)

# Umwandeln in DataFrame zur Visualisierung
df_keywords = pd.DataFrame(keyword_counts.items(), columns=['Keyword', 'Frequency']).sort_values(by='Frequency', ascending=False)
df_keywords.head(10)
    

## 6. Topic Modeling auf Artikeltexten
Mit Topic Modeling können wir zugrunde liegende Themen in den Artikeln identifizieren.

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Textdaten für Topic Modeling vorbereiten
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='german')
text_matrix = vectorizer.fit_transform(df_articles['text'].dropna())

# LDA-Analyse durchführen
lda = LatentDirichletAllocation(n_components=5, random_state=0)
lda.fit(text_matrix)

# Top-Wörter für jedes Thema anzeigen
for index, topic in enumerate(lda.components_):
    print(f"Thema #{index+1}:")
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print("\n")
    

## 7. Visualisierung der Sentiment- und Keyword-Analyse
Abschließend visualisieren wir die Häufigkeit der wichtigsten Schlagwörter sowie die Verteilung der Stimmung in den Artikelbeschreibungen.

In [ ]:

import matplotlib.pyplot as plt

# Top-Schlagwörter plotten
plt.figure(figsize=(10, 6))
df_keywords.head(10).plot(kind='barh', x='Keyword', y='Frequency', legend=False, color='skyblue')
plt.title('Top-Schlagwörter in Artikeln')
plt.xlabel('Häufigkeit')
plt.show()

# Sentiment-Verteilung plotten
sentiment_counts = df_articles['sentiment'].value_counts()
plt.figure(figsize=(6, 6))
sentiment_counts.plot(kind='pie', autopct='%1.1f%%', startangle=140)
plt.title('Sentiment-Verteilung in Artikelbeschreibungen')
plt.show()
    

## Zusammenfassung
In diesem Notebook haben wir verschiedene NLP-Methoden auf einen großen Datensatz von deutschen Nachrichtenartikeln angewendet. Durch Sentiment-Analyse, Named Entity Recognition und Topic Modeling haben wir nützliche Erkenntnisse aus den Artikeldaten gewonnen. Diese Methoden können weiter genutzt werden, um tiefere Analysen in journalistischen Projekten durchzuführen.

**Vielen Dank für die Teilnahme!**